In [114]:
import pandas as pd
from geopy import distance

from FlightRadar24.api import FlightRadar24API
fr_api = FlightRadar24API()

In [2]:
df_airlines = pd.DataFrame(fr_api.get_airlines())[["Name","ICAO"]]
df_airlines.to_parquet("Airlines.parquet")

In [3]:
df_airlines = pd.read_parquet("Airlines.parquet")
df_airlines

,Name,ICAO
0,21 Air,CSB
1,25only Aviation,ONY
2,2Excel Aviation,BRO
3,40-Mile Air,MLA
4,748 Air Services,IHO
...,...,...
1789,Zil Air,SYZ
1790,Zimex Aviation,IMX
1791,Zimex Aviation Austria,AZD
1792,ZIPAIR,TZP


In [105]:
df_airports=pd.read_parquet("Airports.parquet")
df_airports

,name,country,Continent,iata,lat,lon
0,\tMasai Mara Keekorok Airport,Kenya,africa,KEU,-1.586377,35.257462
1,A Coruna Airport,Spain,europe,LCG,43.302059,-8.377250
2,Aachen Merzbruck Airport,Germany,europe,AAH,50.823051,6.186111
3,Aalborg Airport,Denmark,europe,AAL,57.092781,9.849164
4,Aarhus Airport,Denmark,europe,AAR,56.300011,10.619000
...,...,...,...,...,...,...
4936,Zunyi Xinzhou Airport,China,asia,ZYI,27.811167,107.245972
4937,Zurich Airport,Switzerland,europe,ZRH,47.464722,8.549167
4938,Zwara Airport,Libya,africa,WAX,32.933334,12.083333
4939,Zweibrucken Airport,Germany,europe,ZQW,49.209518,7.400647


In [5]:
def active_airplane(a_icao):
    flight_list=[]
    flights = fr_api.get_flights(airline = a_icao)
    for flight in flights :
        try :
            dest_iata = flight.destination_airport_iata
        except :
            dest_iata = None
        try :
            org_iata = flight.origin_airport_iata
        except :
            org_iata = None
        try :
            dest_icao = flight.destination_airport_icao
        except :
            dest_icao = None
        try :
            org_icao = flight.origin_airport_icao
        except :
            org_icao = None
        try :
            model = flight.aircraft_model
        except :
            model = None 
        try :
            registration = flight.registration
        except :
            registration = None
        try :
            speed = flight.ground_speed
        except :
            speed = None

        f = {
            "id" : flight.id,
            "dest_icao" : dest_icao,
            "org_icao" : org_icao,
            "dest_iata" : dest_iata,
            "org_iata" : org_iata,
            "model" : model,
            "registration" : registration,
            "speed" : speed,
            "company" : a_icao
            }
        flight_list.append(f)
    return flight_list

In [6]:
active_flights = []
for airline in df_airlines["ICAO"]:
    active_flights += active_airplane(airline)

In [8]:
df_flights = pd.DataFrame(active_flights)
df_flights = df_flights[df_flights["dest_iata"] != 'N/A']
df_flights = df_flights[df_flights["org_iata"] != 'N/A']
df_flights

,id,dest_icao,org_icao,dest_iata,org_iata,model,registration,speed,company
0,2da0a965,None,None,SBD,ABE,None,N750AX,0,ABX
1,2da0e28b,None,None,LAX,CVG,None,N364CM,378,ABX
2,2da0f3a5,None,None,ABE,SMF,None,N744AX,493,ABX
3,2da16a92,None,None,PHX,CVG,None,N797AX,392,ABX
4,2da1836a,None,None,IAH,CVG,None,N312AA,403,ABX
...,...,...,...,...,...,...,...,...,...
9292,2da18f37,None,None,ARN,GOT,None,OE-HZA,168,IMX
9293,2da18fc7,None,None,ARN,UME,None,HB-ALR,7,IMX
9294,2da0de56,None,None,NRT,BKK,None,JA822J,499,TZP
9295,2da128a4,None,None,NRT,LAX,None,JA826J,515,TZP


In [169]:
df_flights.to_parquet("Flights.parquet")

# Q2

In [170]:
df= pd.read_parquet("Flights.parquet",columns=["dest_iata","org_iata","company"])
airports = pd.read_parquet("Airports.parquet", columns=["iata","Continent"])

def same_continent(x):
    dest = x["dest_iata"]
    org = x["org_iata"]
    try :
        if airports[airports["iata"] == org]["Continent"].iloc[0] == airports[airports["iata"] == dest]["Continent"].iloc[0]:
            return airports[airports["iata"] == org]["Continent"].iloc[0]
        else :
            return None
    except IndexError as e:
        return None

S = df.apply(same_continent, axis=1)
S.sort_values()

5665    africa
3401    africa
3400    africa
3398    africa
3392    africa
         ...  
8267      None
8268      None
8269      None
8274      None
8275      None
Length: 7879, dtype: object

In [60]:
df["Continent"] = df.apply(same_continent, axis=1)

In [61]:
df

,dest_iata,org_iata,company,Continent
0,SBD,ABE,ABX,northamerica
1,LAX,CVG,ABX,northamerica
2,ABE,SMF,ABX,northamerica
3,PHX,CVG,ABX,northamerica
4,IAH,CVG,ABX,northamerica
...,...,...,...,...
9292,ARN,GOT,IMX,europe
9293,ARN,UME,IMX,europe
9294,NRT,BKK,TZP,None
9295,NRT,LAX,TZP,None


In [94]:
for continent in fr_api.get_zones().keys():
    count = df[df["Continent"] == continent]["company"].value_counts()
    if not count.empty:
        print(continent.upper(), ":\n", df_airlines[df_airlines["ICAO"] == count.index[0]].iloc[0]["Name"] , ":", count.max())

EUROPE :
 Ryanair : 272
NORTHAMERICA :
 American Airlines : 771
SOUTHAMERICA :
 LATAM Airlines : 105
OCEANIA :
 Air New Zealand : 49
ASIA :
 IndiGo : 44
AFRICA :
 Binter Canarias : 5
NORTHATLANTIC :
 Air Greenland : 2


In [66]:
fr_api.get_zones().keys()

dict_keys(['europe', 'northamerica', 'southamerica', 'oceania', 'asia', 'africa', 'atlantic', 'maldives', 'northatlantic'])

# Q3

In [174]:
airports = pd.read_parquet("Airports.parquet", columns=["iata","lat","lon"])
airports

,iata,lat,lon
0,KEU,-1.586377,35.257462
1,LCG,43.302059,-8.377250
2,AAH,50.823051,6.186111
3,AAL,57.092781,9.849164
4,AAR,56.300011,10.619000
...,...,...,...
4936,ZYI,27.811167,107.245972
4937,ZRH,47.464722,8.549167
4938,WAX,32.933334,12.083333
4939,ZQW,49.209518,7.400647


In [175]:
df3 = pd.read_parquet("Flights.parquet",columns=["id","dest_iata","org_iata"])
df3

,id,dest_iata,org_iata
0,2da1e3ae,WUT,CAN
1,2da1e58d,NKG,CAN
2,2da1e597,HIA,CAN
3,2da1e715,YIH,CAN
4,2da0e28b,LAX,CVG
...,...,...,...
8271,2da1cab8,UME,ARN
8272,2da1e63d,VXO,ARN
8273,2da1e99d,UME,ARN
8274,2da128a4,NRT,LAX


In [176]:
def airport_dist(x):
    org = x["org_iata"]
    dest = x["dest_iata"]
    try :
        coord_org = (airports[airports["iata"] == org]["lat"].iloc[0], airports[airports["iata"] == org]["lon"].iloc[0])
        coord_dest = (airports[airports["iata"] == dest]["lat"].iloc[0], airports[airports["iata"] == dest]["lon"].iloc[0])
        return distance.distance(coord_org, coord_dest).km
    except IndexError as e:
        return None

In [177]:
S = df3.apply(airport_dist, axis=1)
S.sort_values()

KeyboardInterrupt: 

In [178]:
df3["travel_size"] = df3.apply(airport_dist, axis=1)
df3

,id,dest_iata,org_iata,travel_size
0,2da1e3ae,WUT,CAN,1686.301598
1,2da1e58d,NKG,CAN,1075.589814
2,2da1e597,HIA,CAN,1284.652809
3,2da1e715,YIH,CAN,814.034410
4,2da0e28b,LAX,CVG,3057.418792
...,...,...,...,...
8271,2da1cab8,UME,ARN,477.439941
8272,2da1e63d,VXO,ARN,357.076617
8273,2da1e99d,UME,ARN,477.439941
8274,2da128a4,NRT,LAX,8772.806823


In [179]:
df3.max()

id                 2da1ec9f
dest_iata               ZRH
org_iata                ZTH
travel_size    15348.640441
dtype: object

In [180]:
print("The flight with the longuest journey started at",end=" ")
print(df_airports[df_airports["iata"] == df3.max()["org_iata"]].iloc[0]["name"],end=" ")
print("and will end at",end=" ")
print(df_airports[df_airports["iata"] == df3.max()["dest_iata"]].iloc[0]["name"],end=" ")
print(f'for a journey of {df3.max()["travel_size"]} km')

The flight with the longuest journey started at Zakynthos International Airport and will end at Zurich Airport for a journey of 15348.64044117391 km


# Q4

In [181]:
airports = pd.read_parquet("Airports.parquet", columns=["iata","Continent","lon","lat"])
airports

,iata,Continent,lon,lat
0,KEU,africa,35.257462,-1.586377
1,LCG,europe,-8.377250,43.302059
2,AAH,europe,6.186111,50.823051
3,AAL,europe,9.849164,57.092781
4,AAR,europe,10.619000,56.300011
...,...,...,...,...
4936,ZYI,asia,107.245972,27.811167
4937,ZRH,europe,8.549167,47.464722
4938,WAX,africa,12.083333,32.933334
4939,ZQW,europe,7.400647,49.209518


In [184]:
def cont_org(x):
    try :
        return airports[airports["iata"] == x["org_iata"]]["Continent"].iloc[0]
    except IndexError:
        return None

In [185]:
df4 = pd.read_parquet("Flights.parquet",columns=["id","dest_iata","org_iata"])
df4["travel_size"] = df4.apply(airport_dist, axis=1)
df4["org_Continent"] = df4.apply(cont_org, axis=1)
print("The average route distance is :")
for continent in fr_api.get_zones().keys():
    print(continent.upper(), ":\n", df4[df4["org_Continent"] == continent]["travel_size"].mean(), "km")

The average route distance is :
EUROPE :
 3892.698418604185 km
NORTHAMERICA :
 2325.180607159943 km
SOUTHAMERICA :
 2274.7863026627806 km
OCEANIA :
 2728.6790257987423 km
ASIA :
 3391.4899423756565 km
AFRICA :
 3953.9998775971667 km
ATLANTIC :
 nan km
MALDIVES :
 nan km
NORTHATLANTIC :
 4080.7708056083725 km


# Q5